# Moving Pre Pre-alginment workflow

I have been refactoring the pre-pre and pre-aln workflows. Now I want to move the output over to its new location. 

In [1]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-10-24 
Git hash: 775d8a52b8de98c4320e0ea3917ae4d2a97809cf


In [2]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra2']
remap = db['remap']


In [3]:
import shutil
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
store = pd.HDFStore('../../output/sra.h5', mode='r')

In [5]:
def move(fname, srx=None, srr=None):
    orig = os.path.join('../../output/prealignment/raw/{srx}/{srr}', fname).format(srx=srx, srr=srr)
    new = os.path.join('../../output/prealn-wf/samples/{srx}/{srr}', fname).format(srx=srx, srr=srr)
    
    try:
        shutil.move(orig, new)
    except FileNotFoundError as e:
        pass

In [11]:
patterns = [
    'LAYOUT',
    '{srr}_1.fastq_screen.txt',
    '{srr}_1.fastq_screen.txt.log',
    '{srr}_1.trim.clean.fastq.gz.log',
    '{srr}.fastq.tsv',
    '{srr}.hisat2.bam.bamtools.stats',
    '{srr}.hisat2.bam.feature_counts.counts',
    '{srr}.hisat2.bam.feature_counts.counts.log',
    '{srr}.hisat2.bam.feature_counts.counts.jcounts',
    '{srr}.hisat2.bam.feature_counts.counts.summary',
    '{srr}.hisat2.bam.FIRST_READ_TRANSCRIPTION_STRAND.picard.collectrnaseqmetrics',
    '{srr}.hisat2.bam.FIRST_READ_TRANSCRIPTION_STRAND.picard.collectrnaseqmetrics.log',
    '{srr}.hisat2.bam.log',
    '{srr}.hisat2.bam.NONE.picard.collectrnaseqmetrics',
    '{srr}.hisat2.bam.NONE.picard.collectrnaseqmetrics.log',
    '{srr}.hisat2.bam.picard.markduplicates.metrics',
    '{srr}.hisat2.bam.picard.markduplicates.metrics.log',
    '{srr}.hisat2.bam.samtools.idxstats',
    '{srr}.hisat2.bam.samtools.stats',
    '{srr}.hisat2.bam.SECOND_READ_TRANSCRIPTION_STRAND.picard.collectrnaseqmetrics',
    '{srr}.hisat2.bam.SECOND_READ_TRANSCRIPTION_STRAND.picard.collectrnaseqmetrics.log',
    '{srr}.hisat2.bam.tsv',
    'STRAND',
    'ALIGNMENT_BAD'
]

In [37]:
dfs = []
for i, record in store['ids'].iterrows():
    if os.path.exists('../../output/prealignment/raw/{srx}/{srr}'.format(**record.to_dict())):
        for p in patterns:
            dfs.append(delayed(move)(p, **record.to_dict()))

In [38]:
len(dfs)

562442

In [40]:
with ProgressBar():
    compute(*dfs)

[########################################] | 100% Completed | 47min  9.4s
